In [1]:
%reload_ext autoreload
%autoreload 2

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from collections import OrderedDict
import numpy as np
import re
import h5py
from functools import partial
from itertools import product

from calibration import Display


# Wokflow

* General set up (Tab 0):
   
   - Module Number: Specify channel/module numbers that you want to analyse. Must be comma separated.
   - Pulse indices: Pulses that have x-rays. You can provide a range (start:stop:step) or comma separated (1,3,4 etc) or combination of range and individual pulses (1:10:2, 11, 14, 16 etc)
   - Region of interest (ROI x) in x for each module: Only range based (start:stop)
   - Region of interest (ROI y) in y for each module: Only range based (start:stop)
   
* Dark Run (Tab 1):
    
    - Dark Run Folder: (str) /gpfs/exfel/exp/MID/201931/proposal/raw/run_number
    - Train ids: Train indices to take average over. Range based (start:stop). Deafult is all train (:)
    - Process Darak button: To evalute average. This is done in parallel over modules. Doesn't block
      further analysis. Once results (average image and histograms) are available it be displayed automatically.
    - Using Pulses (slider) and module numbers (dropbox) on top one can visulaize data for each pulses or modules.

* Data Visualization (Tab 2):
    
    - Run Folder: (str) /gpfs/exfel/exp/MID/201931/proposal/raw/run_number
    - Train ids: Train indices. Range based (start:stop). Use cautiosly. It loads all train data in memory.
    Also processing is parallized over modules, therefore maximum pickle size has to be respected.
    
    - Subtract Dark. Once dark average data is available you will be able to check on subtract dark.
    - Load Run: Once things are set up, load the run.
    
    - Fitting Procedure:
    
        - First chose some reasonable peak threshold to filter out number of peaks
        - Chose peak distance to remove very close peaks,
        - Once peaks are chosen, one can click on button Fit Histogram to fit Gaussians. Number of peaks define the number of gaussian functions that will be used to fit the histogram. Generally fitting with 3-4 peaks give reasonable fitting 

In [2]:
config = dict(
    dark_run_folder='/gpfs/exfel/exp/MID/201931/p900091/raw/r0504',
    run_folder='/gpfs/exfel/exp/MID/201931/p900091/raw/r0491',
)

In [3]:
d = Display(config=config)
d.control_panel()

Output(layout=Layout(border='1px solid', height='160px', width='100%'))

# Analysis without GUI



In [21]:
from scipy.ndimage import gaussian_filter, gaussian_filter1d
from scipy.signal import find_peaks
from functools import partial

from calibration import DataProcessing, gauss_fit
from karabo_data import by_index

    
    def DataProcessing(module_number, path, *,
                       train_index=None, pulse_ids=None,
                       rois=None, operation=None,
                       dark_run=None):
                   
           """ Process Data

                    Parameters
                    ----------
                    module_number: int
                        Channel number between 0, 15
                    path: str
                        Path to Run folder
                    train_index: karabo_data (by_index)
                        Default (all trains by_index[:])
                    pulse_ids: str
                        For eg. ":" to select all pulses in a train
                                "start:stop:step" to select indices with certain step size
                                "1,2,3" comma separated pulse index to select specific pulses
                                "1,2,3, 5:10" mix of above two
                        Default: all pulses ":"
                    rois: karabo_data slice constructor by_index
                        Select ROI of image data. For eg. by_index[..., 0:128, 0:64]
                        See karabo_data method: `get_array`

                    operation: function
                        For eg. functools.partial(np.mean, axis=0) to take mean over trains
                    dark_run: nd.array
                        dark_data to subtract

                    Return
                    ------
                    out: ndarray
                        Shape:  operation -> (n_trains, n_pulses, ..., slow_scan, fast_scan)
                    """

# General Parameters

In [29]:
module_number = 15
pulse_ids = "1:250:2"
rois = by_index[:, :]

# Dark Run average

In [24]:
# parameters

dark_run_folder = "/gpfs/exfel/exp/MID/201931/p900091/raw/r0504"

module_number = 15

dark_train_index = by_index[:]


# mean over train index 
operation = partial(np.mean, axis=0)



In [26]:
dark_average = DataProcessing(
    module_number, 
    dark_run_folder,
    train_index=dark_train_index,
    pulse_ids=pulse_ids,
    rois=rois,
    operation=operation)

print(dark_average.shape)

(125, 2, 512, 128)


# Subtract Dark from a run

In [31]:

run_folder = "/gpfs/exfel/exp/MID/201931/p900091/raw/r0491"

proc_train_index = by_index[250:280]

dark_run = {module_number:dark_average}

In [32]:
dark_subtracted = DataProcessing(
    module_number, 
    run_folder,
    train_index=proc_train_index,
    pulse_ids=pulse_ids,
    rois=rois,
    dark_run=dark_run)

In [33]:
print(dark_subtracted.shape)

(30, 125, 2, 512, 128)
